In [1]:
from util import load_user_data, task_ID_list_to_check
import pandas as pd
import numpy as np

valid_users, tp_data = load_user_data(folder_name="../anonymized_data", reserved_users=None)
user2condition = tp_data['user2condition']
user_planning_actions = tp_data['user_planning_actions']
condition_count = {}
for user in valid_users:
    tp_condition = user2condition[user]
    if tp_condition not in condition_count:
        condition_count[tp_condition] = 0
    condition_count[tp_condition] += 1
print(condition_count)

from util import load_data
filename = "../simplified_example_original.json"
task_ID_list, planning_task_dict = load_data(filename)


248 complete the NASA-TLX
248 complete the whole study
{'UP-UE': 60, 'AP-UE': 64, 'AP-AE': 63, 'UP-AE': 61}


In [2]:
for user in valid_users:
    if user2condition[user] in ["AP-AE", "AP-UE"]:
        continue
    if user not in user_planning_actions:
        print(f"{user} does not contain any planning edits")
        continue
    for task_id in task_ID_list_to_check:
        if task_id not in user_planning_actions[user]:
            print(f"{user} does not contain planning edits for task {task_id}")

In [3]:
def compare_plan(plan_1, plan_2):
    if len(plan_1) != len(plan_2):
        return False
    length = len(plan_1)
    for index in range(length):
        step_1 = plan_1[index]
        step_2 = plan_2[index]
        step_text_1 = step_1['step'].strip()
        step_text_2 = step_2['step'].strip()
        if step_text_1 == step_text_2 and step_1['index'] == step_2['index']:
            pass
        else:
            return False
    # Both length and all steps are the same
    return True

def check_plan_new(tp_plan_str):
    tp_plan = tp_plan_str.strip().split("\n")
    new_plan = []
    for step in tp_plan:
        # plan_text = step["step"]
        plan_text = step
        index = plan_text.split(" ")[0]
        plan_text = plan_text[len(index)+1:]
        if len(index.split(".")) == 3:
            # x.y.z
            data_level = 3
        elif index[-1] == ".":
            # x.
            data_level = 1
        else:
            # x.y
            data_level = 2
        tp_obj = {
            "step": plan_text,
            "data_level": data_level,
            "index": index
            # "tool": step["tool"]
        }
        new_plan.append(tp_obj)
    return new_plan


In [4]:
import json
diff_user_set = set()
tp_user_set = set()
number_user_task = 0
action_dict = {}
user_task_plan = tp_data['user_task_plan']
task_edit_freq = {}
for user in valid_users:
    if user2condition[user] in ["AP-AE", "AP-UE"]:
        continue
    for task_id in task_ID_list_to_check:
        planning_actions = user_planning_actions[user][task_id]
        if isinstance(planning_actions, str):
            tp_plan = check_plan_new(user_task_plan[user][task_id])
            original_plan = planning_task_dict[task_id].plan
            flag = compare_plan(plan_1=original_plan, plan_2=tp_plan)
            if not flag:
                diff_user_set.add(user)
                if task_id not in task_edit_freq:
                    task_edit_freq[task_id] = 0
                task_edit_freq[task_id] += 1
            planning_actions = json.loads(planning_actions)
            assert isinstance(planning_actions, list)
            # if len(planning_actions) == 0:
            #     print(f"{user} make no add/split/delete on task {task_id}")
            if len(planning_actions) > 0:
                # print(user, task_id, planning_actions)
                flag = False
                for action in planning_actions:
                    action_name = action['action_event']
                    if action_name in ["show_potential_actions", 'action_prediction', 'Proceed', 'execute_action', 'Next Step']:
                        continue
                    if action_name not in action_dict:
                        action_dict[action_name] = 0
                    action_dict[action_name] += 1
                    flag = True
                if flag:
                    tp_user_set.add(user)
                    number_user_task += 1
        else:
            assert isinstance(planning_actions, list)
            if len(planning_actions) == 0:
                print(f"{user} make no add/split/delete on task {task_id}")
            else:
                print("Unknown case", planning_actions)
print(f"{len(diff_user_set)} users edit the plan with text edit or buttons")
print(f"{len(tp_user_set)} users edit the plan with provided buttons")
print(f"{number_user_task} times edit the plan with provided buttons")
print(action_dict)
print(task_edit_freq)

104 users edit the plan with text edit or buttons
90 users edit the plan with provided buttons
241 times edit the plan with provided buttons
{'plan|delete_step': 394, 'plan|add_step': 183, 'plan|split_step': 126}
{'test-149': 60, 'test-200': 62, 'test-675': 48, 'test-859': 51, 'test-388': 44, 'test-497': 45}
